In [1]:
%pip install qiskit --quiet
%pip install qiskit-aer --quiet
%pip install pylatexenc --quiet
%pip install qiskit-machine-learning --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 102.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 12.7 MB/s eta 0:00:00


In [2]:
%pip install qiskit-algorithms --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 15.6 MB/s eta 0:00:00


In [3]:
import math
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import os

from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tensorflow import keras

import qiskit
from qiskit import QuantumCircuit, transpile
from qiskit.transpiler import generate_preset_pass_manager
from qiskit.visualization import plot_histogram
from qiskit.circuit.library import RealAmplitudes, EfficientSU2, ZZFeatureMap, TwoLocal
import pylatexenc

In [35]:
import torch.nn as nn
from torch.nn import Linear, CrossEntropyLoss, MSELoss
from torch.optim import LBFGS

from qiskit.circuit import Parameter
from qiskit_algorithms.utils import algorithm_globals
from qiskit_machine_learning.neural_networks import SamplerQNN, EstimatorQNN
from qiskit_machine_learning.connectors import TorchConnector

from qiskit.quantum_info import Statevector
from qiskit.primitives import Sampler
from qiskit_algorithms.state_fidelities import ComputeUncompute
from qiskit_machine_learning.kernels import FidelityQuantumKernel

import torch
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Datasets

##Medquad (HCD) dataset

In [6]:
max_length=51
df = pd.read_csv(r'/content/drive/MyDrive/LLM Healthcare/medquad.csv', usecols=['question', 'focus_area'])
arr=df.to_numpy()
# Label count and index
label_count={}
for i in arr:
  label_count[i[-1]]=label_count.get(i[-1],0)+1
labels_count=sorted(label_count.items(),key=lambda x:-x[1])
label_index={i[0]:j for j,i in enumerate(labels_count)}
#Word count and index
word_counts={}
for i in arr:
  for j in i[0].split():
    word_counts[j]=word_counts.get(j,0)+1
word_counts=sorted(word_counts.items(),key=lambda x:-x[1])
word_index={i[0]:j+1 for j,i in enumerate(word_counts)}
word_index['PAD']=0
word_index=dict(sorted(word_index.items(),key=lambda x:x[1]))
#Tokenization
def tokenization(words):
  token=[]
  for i in words:
    k=[]
    for j in i.split():
      k.append(word_index[j])
    token.append(k)
  return token
# Tokenizing the data
tokenized_data=tokenization([i[0] for i in arr])
labels=[label_index[i[1]] for i in arr]
#Padding the data
data= keras.preprocessing.sequence.pad_sequences(tokenized_data,maxlen=max_length,padding="post")
#Only selecting two labels
new_data,label=[],[]
for i,j in zip(data,labels):
  if j<2:
    new_data.append(i)
    label.append(j)
combined_data=pd.DataFrame(np.c_[new_data,label])
h_data=new_data
h_label= label
combined_data.columns = list(combined_data.columns)[:-1] + ['Cluster']

In [7]:
df = combined_data
cluster = np.array(df['Cluster'])

In [8]:
numeric_cols = df.select_dtypes(include=np.number).columns
all_zero_cols = [col for col in numeric_cols if df[col].sum() == 0]
df_cleaned = df.drop(columns=all_zero_cols)

In [9]:
df = df_cleaned

In [36]:
row_arrays = []
for index, row in df.iterrows():
    row_arrays.append(row.to_numpy())

In [11]:
h_data = row_arrays

In [34]:
def create_qnn():
    feature_map = ZZFeatureMap(2, reps= 3)
    ansatz = RealAmplitudes(2, reps=3)

    ansatz = ansatz.assign_parameters({param: Parameter(f'phi[{i}]') for i, param in enumerate(ansatz.parameters)})

    qc = QuantumCircuit(2)

    qc.h(0)
    qc.h(1)

    qc.compose(feature_map, inplace=True)

    qc.compose(ansatz, inplace=True)

    qc.h(0)
    qc.h(1)

    qnn = EstimatorQNN(
        circuit=qc,
        input_params=feature_map.parameters,
        weight_params=ansatz.parameters,
        input_gradients=True,
    )
    return qnn

In [33]:
qnn4 = create_qnn()

vocab_size_h=118
import torch
import torch.nn as nn
from torch.nn import functional as F
import torch
import torch.nn as nn
from qiskit_machine_learning.connectors import TorchConnector

class ImprovedNN(nn.Module):
    def __init__(self, vocab_size_h, qnn4):
        super().__init__()

        self.embedding_dim = 16
        self.sequence_length = 13

        self.embed = nn.Embedding(vocab_size_h, self.embedding_dim)

        encoder_input_size = self.sequence_length * self.embedding_dim

        self.encoder = nn.Sequential(
            nn.Linear(encoder_input_size, 128),
            nn.ReLU(),
            nn.LayerNorm(128),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.LayerNorm(64),
            nn.Linear(64, 2),
            nn.Tanh()
        )

        self.qnn = TorchConnector(qnn4)
        self.post_qnn = nn.Sequential(
            nn.Linear(1, 1),
            nn.Sigmoid()                 # Binary classification
        )

    def forward(self, x):
        x = self.embed(x)
        x = x.view(x.size(0), -1)

        x = self.encoder(x)
        x = self.qnn(x)

        x = self.post_qnn(x)                   # Final classifier layer
        return x.view(-1)

/tmp/ipython-input-2108818143.py:26: DeprecationWarning: V1 Primitives are deprecated as of qiskit-machine-learning 0.8.0 and will be removed no sooner than 4 months after the release date. Use V2 primitives for continued compatibility and support.
  qnn = EstimatorQNN(


In [30]:
# Assuming h_data and h_label are numpy arrays
m_data_tensor = torch.tensor(h_data, dtype=torch.long)
m_data_labels = torch.tensor(h_label, dtype=torch.float32)

X_train, X_val, y_train, y_val = train_test_split(
    m_data_tensor,
    m_data_labels,
    test_size=0.2,       # 20% validation
    shuffle=True,
    random_state=42
)

epochs = 50
loss_fn = torch.nn.BCELoss()
loss_list = []
model = ImprovedNN(vocab_size_h, qnn4)
optimizer = torch.optim.Adadelta(model.parameters(), lr=0.01, weight_decay=0.8)

for epoch in range(epochs):
    print(f'For epoch: {epoch+1}/{epochs}')
    model.train()
    optimizer.zero_grad()
    output = model(X_train)
    loss = loss_fn(output, y_train)
    loss_list.append(loss)
    loss.backward()
    optimizer.step()

    with torch.no_grad():
        acc = (output > 0.5).int().eq(y_train.int()).float().mean().item()
        if acc == 1.0:
            break

model.eval()
with torch.no_grad():
    preds = (model(X_val) > 0.5).int()
    y_val_np = y_val.int().numpy()
    preds_np = preds.numpy()

    accuracy = accuracy_score(y_val_np, preds_np)
    precision = precision_score(y_val_np, preds_np)
    recall = recall_score(y_val_np, preds_np)
    f1 = f1_score(y_val_np, preds_np)


print(f"Accuracy : {accuracy}")
print(f"Precision: {precision}")
print(f"Recall   : {recall}")
print(f"F1 Score : {f1}")

For epoch: 1/50
For epoch: 2/50
For epoch: 3/50
For epoch: 4/50
For epoch: 5/50
For epoch: 6/50
For epoch: 7/50
For epoch: 8/50
For epoch: 9/50
For epoch: 10/50
For epoch: 11/50
For epoch: 12/50
For epoch: 13/50
For epoch: 14/50
For epoch: 15/50
For epoch: 16/50
For epoch: 17/50
For epoch: 18/50
For epoch: 19/50
For epoch: 20/50
For epoch: 21/50
For epoch: 22/50
For epoch: 23/50
For epoch: 24/50
For epoch: 25/50
For epoch: 26/50
For epoch: 27/50
For epoch: 28/50
For epoch: 29/50
For epoch: 30/50
For epoch: 31/50
For epoch: 32/50
For epoch: 33/50
For epoch: 34/50
For epoch: 35/50
For epoch: 36/50
For epoch: 37/50
For epoch: 38/50
For epoch: 39/50
For epoch: 40/50
For epoch: 41/50
For epoch: 42/50
For epoch: 43/50
For epoch: 44/50
For epoch: 45/50
For epoch: 46/50
For epoch: 47/50
For epoch: 48/50
For epoch: 49/50
For epoch: 50/50
Accuracy : 0.95
Precision: 0.9090909090909091
Recall   : 1.0
F1 Score : 0.9523809523809523
